In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
from datetime import datetime
import math

In [2]:
# Importing the excel
dataset = pd.read_excel('punch_short.xlsx')

In [3]:
#Renaming column name to a valid column name format as per python
dataset.rename(columns={'IN/OUT': 'IN_OUT'}, inplace=True)

In [4]:
#Droping rows that conatain 1 in 'IN_OUT' column
dataset=dataset[dataset.IN_OUT!=1]

In [ ]:
#Converting Date column values into year-month-day format
j,k=0,0
for i in dataset.iloc[:,3]:
  dataset.Date[j]=datetime.strptime(str(i),'%Y%m%d').date()
  j+=1

dataset['Date'] = pd.to_datetime(dataset['Date'], errors='coerce')

In [6]:
#Calculating and appending Punch time difference for each employee with the corresponding E.Code into dataframe (l)
l=[]
for i in range(len(dataset)-1):
  j=i+1
  l.append([dataset.iloc[i]['E.Code'], datetime.combine(dataset.Date[0], datetime.strptime(str(dataset.iloc[j]['Time']),'%H%M%S').time()) - datetime.combine(dataset.Date[0], datetime.strptime(str(dataset.iloc[i]['Time']),'%H%M%S').time())])  
  i=i+2

In [7]:
#Inserting date in datetime format after E.Code column
for i in range(len(l)):
  l[i].insert(1,dataset.iloc[i]['Date'].date())

In [8]:
#Replacing ":" with "." and getting upto 2 decimal places in Time to make mathematical operations feasible on i
for i in range(len(l)):
  l[i][2] = l[i][2].__str__().replace(":",".")
  s=l[i][2]
  l[i][2]=s[0:-3]

In [9]:
#Removing redundant rows and storing in new data-structure
l2=[]
for i in range(len(l)):
  if 'd' not in str(l[i][2]):
    l2.append(l[i]) 

In [10]:
#Converting string type to float for Time column
for i in range(len(l2)):
  l2[i][2]=float("{:.2f}".format(float(l2[i][2])))

In [11]:
"""Converting time from float to mins, applied logic to check if it is greater than 9hrs & 30mins. Subtacting the 
result to get over time (minutes) converting and storing it into hrs.minutes format. """

l3=[]
for i in range(len(l2)):
    frac, whole = math.modf(l2[i][2])
    mins=int(whole*60+frac*100)
    if mins>570:
      mins=mins-570
      hours=mins//60
      minutes=mins%60
      if minutes<10:
        ot_time = float("{}.0{}".format(hours, minutes))
      else:
        ot_time = float("{}.{}".format(hours, minutes))
      l2[i].append(ot_time)
      l3.append(l2[i])

In [14]:
# Converting 2d list to dataframe to apply CRUD opertations
df = pd.DataFrame(l3, columns=['E.Code','Date','Total Time','OT'])

In [15]:
#Droping an unnecesary column
df=df.drop(['Total Time'], axis=1)

In [ ]:
""" Printing dataframe (E.Code, Date, OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes)) 
"""

df

In [17]:
""" Applying Groupby opr. to group dataframe on basis of E.Code and apply sum and count opr. to 
compute Total_OT & Total_OT_Days """

groupby_ecode_Total_OT = df.groupby(['E.Code']).sum()
groupby_ecode_Total_OT_Days = df.groupby(['E.Code']).count()

In [18]:
#Converting Groupby object into DataFrame to apply CRUD opr.
df2=pd.DataFrame(groupby_ecode_Total_OT)
df3=pd.DataFrame(groupby_ecode_Total_OT_Days)

In [20]:
l3=[]
for i in range(len(df2)):
    frac, whole = math.modf(df2.iloc[i]['OT'])
    mins=int(whole*60+frac*100)
    hours=mins//60
    minutes=mins%60
    if minutes<10:
      ot_time = float("{}.0{}".format(hours, minutes))
    else:
      ot_time = float("{}.{}".format(hours, minutes))
    l3.append(ot_time)

In [21]:
df2['Total OT']=l3

In [22]:
df2=df2.drop(['OT'], axis=1)

In [23]:
#Droping an unnecesary column
df3=df3.drop(['Date'], axis=1)

In [24]:
#Renaming Columns
# df2.rename(columns={'OT': 'Total OT'}, inplace=True)
df3.rename(columns={'OT': 'Total OT_Days'}, inplace=True)

In [25]:
#Creating a final dataframe (E.Code, Total OT_Days, Total OT)
result = pd.concat([df2,df3], axis=1)

In [ ]:
""" Printing dataframe (E.Code, Total OT (after decimal place refers to mins, before refers to hrs; 
e.g 0.27--> 27 minutes, 
    1.08--> 1hr & 8 minutes), 
    Total OT_Days) 
"""
result

In [27]:
# Sorting dataframe in descending order on basis of Total OT_Days (User can get top x employees on basis of Total OT_Days)
context2 = result.sort_values(by=['Total OT_Days'], ascending=False)

In [28]:
# Sorting dataframe in descending order on basis of Total OT (User can get top x employees on basis of Total OT)
context3 = result.sort_values(by=['Total OT'], ascending=False)

In [ ]:
!pip install xlsxwriter

In [ ]:
#from openpyxl import Workbook
import xlsxwriter
workbook = xlsxwriter.Workbook('dd.xlsx')

In [ ]:
with pd.ExcelWriter('dd.xlsx') as writer:
                df.to_excel(writer,sheet_name='OT')
                result.to_excel(writer,sheet_name='Total_OT')
                context2.to_excel(writer,sheet_name='Descending_Total_OT_Days')
                context3.to_excel(writer,sheet_name='Descending_Total_OT')